# City/County GDP Forecasting (Omaha Test)
A script that will clean and organize the data, preform a bootstrap regression to identify significant variables and their effect on the GDP. Then using the regression results, conduct Monte Carlo Simulations on all the different economic categories to predict the distribution of likely results going forward.

##TO DO
*   Hyper Parameter Tuning
*   Model Selection
*   Create models for labor predictions
*   Save predictions to a database (NoSQL?)
*   Create a monte carlo graph from the models
*   Automate data retrieval
*   Feature Engineering (ALWAYS BE DOING)
    * Consider taking the natural log of our Y value

Import the standard libraries for data analysis

In [34]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn import svm
from sklearn.metrics import mean_squared_error
import json
import requests
import datetime
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
import math
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from datetime import date
import math
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import ElasticNet
from numpy import arange
from sklearn.linear_model import ElasticNetCV
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from numpy import asarray
from sklearn.preprocessing import MinMaxScaler

Create a dictionary that contains all of the states and their abbreviations. We will use this to convert variables and make it possible to join different datasets.

In [35]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

#Swap the value and keys
us_states = {value:key for key, value in us_state_to_abbrev.items()}

Read in the data gathered from the bureau of economic analysis(BEA). The data was specifically the form 'CAGDP9' which can be found at "https://apps.bea.gov/iTable/iTable.cfm?reqid=99&step=1#reqid=99&step=1 "

In [36]:
data = pd.read_csv('/content/drive/MyDrive/City Forecasting Website/CAGDP9__ALL_AREAS_2001_2020.csv', encoding='cp1252')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [37]:
#Change the vaalue name 'United States' to 'U.S. TOTAL' so that we can comabine it with the BLS/QWEC data
data['GeoName'] = data['GeoName'].replace(r'United States', 'U.S. TOTAL', regex=True)

In [38]:
#Swap out the state names with the state abbreviations using the dictionary we created
temp = []

def replace_words(s, words):
    for k, v in words.items():
        s = s.replace(', ' + k,', ' + v)
    return s

for i in data['GeoName']:
  temp.append(replace_words(str(i), us_states))

data['GeoName'] = temp

In [39]:
# Remove all white space and make all letters lower case to make joining easier
temp = []

for i in data['GeoName']:
  x = i.replace(" ", "")
  temp.append(x.lower())

data['GeoName'] = temp

# 1. Tidy and prep the data so that we are able to conduct a regression

Create a function which will pull in all of the csv files from the CEW which have been saved as objects. TODO: Continue to try and identify a series ID will will let us pull in this data through the API.

In [40]:
def bls(year = 2000):
  # Data can be found here https://www.bls.gov/cew/downloadable-data-files.htm
  # Data descriptions can be found here: https://www.bls.gov/cew/about-data/downloadable-file-layouts/annual/naics-based-annual-layout.htm
  employ_data = pd.read_csv('/content/drive/MyDrive/City Forecasting Website/QCEW/{}.annual 10 Total, all industries.csv'.format(year))
  #Only select the overall total, not governmetn vs private sector jobs
  employ_data = employ_data[employ_data['own_title'] == 'Total Covered']
  
  employ_data['year'] = year

  employ_data = employ_data[['area_title', 'year', 'annual_avg_estabs_count', 'annual_avg_emplvl', 'avg_annual_pay', 'oty_annual_avg_estabs_count_pct_chg', 'oty_annual_avg_emplvl_pct_chg','oty_avg_annual_pay_pct_chg']]

  temp = []

  #Remove the word county from the location as well as all the white space and make them lower case to join with the BEA data
  for i in employ_data['area_title']:
    x = i.replace("County", "")
    x = x.replace(" ", "")
    temp.append(x.lower())

  employ_data['area_title'] = temp

  #Continue to call all of the csv files and concate them all into one datframe
  try:
    year = year + 1
    employ_data = pd.concat([employ_data, bls(year)], ignore_index=True)
    return(employ_data)
  except:
    return(employ_data)


In [41]:
#call the function and save the data to our variable
employ_data = bls()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.


In [42]:
temp = []

#Convert all the county names into a format that we can join between BLS and BEA
for i in employ_data['area_title']:
  #Remove the word county from the strings
  x = i.replace("County", "")
  #Remove all the whitespace
  x = x.replace(" ", "")
  #Force all the letters into lowercase
  temp.append(x.lower())

#Overwrite the old data with the new data
employ_data['area_title'] = temp

"Melt" or "Gather" the data so that all the years and gdp values are their own columns.

In [43]:
#get all of the data in a formate which has a column for year
data = pd.melt(data, id_vars=['GeoFIPS', 'GeoName', 'Region', 'TableName', 'LineCode',
       'IndustryClassification', 'Description', 'Unit'])

In [44]:
#Convert the data to numerics that we can fit model to
data['value'] = pd.to_numeric(data['value'], errors = 'coerce')
data = data.rename(columns = {'variable':'year', 'value':'GDP'})
data['year'] = pd.to_numeric(data['year'], errors = 'coerce')

In [45]:
#Merge the BEA data and the BLS data based on county name and year
data = data.merge(employ_data, how = 'left', left_on = ['GeoName','year'], right_on = ['area_title', 'year'])

In [46]:
#Remove any rows that are missing data
data = data.dropna()

group by the type of business category and then by the year (variable) to get them in the correct order. Then create a new column with all of the gdp values shifted down to get a variable that represents prior periods gdp. Then remove the first year(2001) in each group since those won't have a previous gdp value

In [47]:
#Create a dataframe with all of the variables that are important in our predictions
blah = data[['year', 'Description','GeoName','GDP','annual_avg_estabs_count', 'annual_avg_emplvl',
                          'avg_annual_pay', 'oty_annual_avg_estabs_count_pct_chg',
                          'oty_annual_avg_emplvl_pct_chg', 'oty_avg_annual_pay_pct_chg']]

#Sort by county and year, this will be important since we will need to shift the data to ensure we are creating models on the previous year's data
blah = blah.sort_values(by = ['GeoName', 'year'])

#Shift all of the variables by 1 year, and include another variable for gdp being shifted 2 years
blah['prevGDP'] = blah['GDP'].shift(periods = 1)
blah['prevGDP2'] = blah['GDP'].shift(periods = 2)
blah['annual_avg_estabs_count'] = blah['annual_avg_estabs_count'].shift(periods = 1)
blah['annual_avg_emplvl'] = blah['annual_avg_emplvl'].shift(periods = 1)
blah['avg_annual_pay'] = blah['avg_annual_pay'].shift(periods = 1)
blah['oty_annual_avg_estabs_count_pct_chg'] = blah['oty_annual_avg_estabs_count_pct_chg'].shift(periods = 1)
blah['oty_annual_avg_emplvl_pct_chg'] = blah['oty_annual_avg_emplvl_pct_chg'].shift(periods = 1)
blah['oty_avg_annual_pay_pct_chg'] = blah['oty_avg_annual_pay_pct_chg'].shift(periods = 1)

#Remove the years which have missing data due to shifted data
#Also remove 2020 because how to you model exogenous shock?
blah = blah[~blah['year'].isin(['2001', '2002', '2020'])]

#Right now we are only interested in the entire couty's prediction
#Create a function to cycle through all of the other subsectors
blah = blah[blah['Description'] == 'All industry total']
blah = blah.dropna()

## 2. Build the Models

Breake out the dataset into the appropriate subsets so that we can run the model. We will create multiple models and generate an ensemble model from them to create a best prediction.

In [48]:
#Assign our Y variable (GDP) and normalize by taking the log
Y = pd.to_numeric(np.log(blah['GDP']),errors='coerce')

Create dummy variables for the gdp category (description) and drop the first one to prevent perfect multicollinearity

In [49]:
#Assign the scaler function so that we can normalize all of the large continuous data
scaler = MinMaxScaler()

#Create a dataframe and assign all of the shift variables we calculated earlier
X = blah[['prevGDP', 'annual_avg_estabs_count', 'avg_annual_pay','annual_avg_emplvl', 
          'oty_annual_avg_estabs_count_pct_chg', 'oty_annual_avg_emplvl_pct_chg', 'oty_avg_annual_pay_pct_chg']]

#Scale the data
X = scaler.fit_transform(X)

#Create variables which are squared versions and interacting versions of the variables brought in by the original dataset
poly = PolynomialFeatures(degree=2)
X = pd.DataFrame(poly.fit_transform(X))

In [50]:
#Split the data so that we can test our model on out-of-sample data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [18]:
#def build_rf_model(X_train = X_train, y_train = y_train):
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 110, num = 12)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
# bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
              'max_features': max_features,
              'max_depth': max_depth,
              'min_samples_split': min_samples_split,
              'min_samples_leaf': min_samples_leaf
              }
#pprint(random_grid)

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


KeyboardInterrupt: ignored

Compare the cv identified best parameters model versus a base model

In [ ]:
def evaluate(model, test_features, test_labels):
  predictions = model.predict(X_test)
  errors = abs(predictions - y_test)
  mape = 100 * np.mean(errors / y_test)
  accuracy = 100 - mape
  print('Model Performance')
  print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
  print('Accuracy = {:0.2f}%.'.format(accuracy))

  return(accuracy)

base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)

best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, y_test)

print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

Model Performance
Average Error: 0.1518 degrees.
Accuracy = 98.85%.
Model Performance
Average Error: 0.1406 degrees.
Accuracy = 98.94%.
Improvement of 0.09%.


In [51]:
RFR = RandomForestRegressor().fit(X_test, y_test)
#SVM = svm.SVR().fit(X_test, y_test)
#OLS = linear_model.LinearRegression().fit(X_test, y_test)
#ADA = AdaBoostRegressor(random_state=0, n_estimators=100).fit(X_test, y_test)
#GBR = GradientBoostingRegressor(random_state=0).fit(X_test, y_test)

In [52]:
models = [RFR]

### Determine the Model Weights for ensemble models
Use the models to create predictions and compare them to the true values. Calculate the Mean Squared Errors (MSE) and use them to determine the weight of each of the models when combine their predictions.

In [53]:
MSE = []
from sklearn.metrics import mean_absolute_error

for i in models:
  y_pred = i.predict(X_test)
  y_true = y_test
  MSE.append(mean_absolute_error(y_true, y_pred))

In [54]:
weights = pd.DataFrame(columns = ['model','weight'])
weightslist = []
MSESum = sum(MSE)
temp = []

for i in MSE:
  x = ((MSESum - i) / MSESum)
  weightslist.append(x)

for i in weightslist:
  x = 1#i / sum(weightslist)
  temp.append(x)

weights['model'] = models
weights['weight'] = temp

In [55]:
weights

,model,weight
0,"(DecisionTreeRegressor(max_features='auto', ra...",1


## Create a prediction

Evaluate all the weights of the models and then drop those that are least effective. Maybe at a 5% level?

In [56]:
prediction = pd.DataFrame()
z = 0

for i in models:
  partial_prediction = i.predict(X_test) * weights['weight'].iloc[z]
  prediction[z] = partial_prediction
  z = z + 1

prediction = prediction.sum(axis = 1)

In [83]:
print(mean_squared_error(y_true, prediction))

from sklearn.metrics import r2_score

print(r2_score(y_true, prediction))

0.011840800925261408
0.995395289376654


In [57]:
prediction = pd.DataFrame(prediction)

In [60]:
compare = pd.DataFrame(np.exp(y_true))
compare = compare.reset_index()
compare['predict'] = np.exp(prediction)
compare['difference'] = compare['predict'] - compare['GDP'] 
compare['pctdif'] = ((compare['difference']) / (compare['GDP'] + compare['predict']) / 2) * 100


In [61]:
compare.describe()

,index,GDP,predict,difference,pctdif
count,1.017500e+04,1.017500e+04,1.017500e+04,1.017500e+04,10175.000000
mean,1.131574e+06,1.470771e+07,1.445455e+07,-2.531560e+05,0.060948
std,5.330780e+05,3.937832e+08,3.839924e+08,1.902837e+07,2.535843
min,2.161800e+05,9.563000e+03,1.243453e+04,-1.494997e+09,-30.413033
25%,6.721530e+05,3.341645e+05,3.370370e+05,-2.648172e+04,-0.719579
50%,1.128704e+06,8.568500e+05,8.495751e+05,6.104123e+03,0.253921
75%,1.595489e+06,2.443326e+06,2.384312e+06,3.421397e+04,1.165007
max,2.053366e+06,1.860679e+10,1.711179e+10,7.772547e+08,23.341287


##Monte Carlo Simulation
Run a Monte Carlo simulation for each of the counties

In [62]:
from statistics import mean

temp = []

for i in range(10000):
  temp.append(compare['pctdif'].sample(frac=1, replace=True).std())

county_gdp_std = mean(temp)

In [103]:
#Assign the scaler function so that we can normalize all of the large continuous data
scaler = MinMaxScaler()

#Take the latest years data that we will run our model on to get the next years prediction
total_predictions = data[data['Description'] == 'All industry total']
max_year = total_predictions['year'].max()
total_predictions = total_predictions[total_predictions['year'] == max_year]

#Pull the previous year's data so that we can include an AR(1) of GDP
prevGDP = data[data['year'] == 2019]
prevGDP = prevGDP[prevGDP['Description'] == 'All industry total'].reset_index()
prevGDP = prevGDP[['GeoName', 'GDP']]
prevGDP = prevGDP.rename({'GDP':"prevGDP"}, axis='columns')

total_predictions = total_predictions.merge(prevGDP, how = 'left', on = ['GeoName'])

Counties = total_predictions['GeoName']

total_predictions  = total_predictions[['GDP', 'annual_avg_estabs_count', 'annual_avg_emplvl',
                          'avg_annual_pay', 'oty_annual_avg_estabs_count_pct_chg',
                          'oty_annual_avg_emplvl_pct_chg', 'oty_avg_annual_pay_pct_chg']]

#Scale the data
total_predictions = scaler.fit_transform(total_predictions)

#Create variables which are squared versions and interacting versions of the variables brought in by the original dataset
poly = PolynomialFeatures(degree=2)
total_predictions = pd.DataFrame(poly.fit_transform(total_predictions))

In [104]:
GDP_Predictions = pd.DataFrame(np.exp(RFR.predict(total_predictions)))
GDP_Predictions = pd.concat([GDP_Predictions, Counties], axis = 1)

In [105]:
GDP_Predictions = GDP_Predictions.set_index('GeoName')

In [106]:
GDP_Predictions

,0
GeoName,
u.s.total,8.021368e+09
"autauga,alabama",2.029964e+06
"baldwin,alabama",5.999697e+06
"barbour,alabama",1.690886e+06
"bibb,alabama",1.564386e+06
...,...
"sweetwater,wyoming",3.049010e+06
"teton,wyoming",3.177342e+06
"uinta,wyoming",1.668581e+06


In [107]:
temp = []
temp_pred = []

for j in range(10000):
  temp.append(random.gauss(0, county_gdp_std))


for i in range(len(GDP_Predictions)):
  temp_pred.append(GDP_Predictions[0].iloc[i] * (1 + np.array(temp) / 100))

In [108]:
temp_pred = pd.DataFrame(temp_pred)
temp_pred.insert(0, 'Counties', Counties)

In [109]:
temp_pred

,Counties,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,9960,9961,9962,9963,9964,9965,9966,9967,9968,9969,9970,9971,9972,9973,9974,9975,9976,9977,9978,9979,9980,9981,9982,9983,9984,9985,9986,9987,9988,9989,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,u.s.total,8.145588e+09,8.064685e+09,7.963706e+09,8.289301e+09,7.837760e+09,8.473008e+09,8.189650e+09,8.111934e+09,8.035775e+09,7.797407e+09,7.842902e+09,8.078166e+09,8.483481e+09,8.496631e+09,8.118284e+09,8.161378e+09,8.151312e+09,7.916830e+09,7.655828e+09,8.379785e+09,7.994227e+09,7.778329e+09,8.173407e+09,8.213755e+09,7.999417e+09,8.185026e+09,8.258851e+09,8.126448e+09,7.752985e+09,8.021888e+09,7.878145e+09,8.174134e+09,7.885578e+09,8.039306e+09,7.850445e+09,7.988715e+09,8.086390e+09,7.774856e+09,8.179211e+09,...,8.001143e+09,7.861613e+09,8.381840e+09,8.136211e+09,8.095377e+09,7.753522e+09,7.685287e+09,8.207305e+09,8.045243e+09,8.053514e+09,7.839992e+09,7.922001e+09,8.129661e+09,7.958002e+09,8.066281e+09,8.393966e+09,8.136392e+09,8.054178e+09,8.048403e+09,8.142851e+09,7.729677e+09,7.964336e+09,7.878598e+09,8.091775e+09,7.786455e+09,7.728434e+09,7.941260e+09,8.247397e+09,7.894851e+09,7.869355e+09,8.309403e+09,8.106957e+09,8.152936e+09,8.229406e+09,8.111815e+09,7.985707e+09,8.453817e+09,8.241147e+09,8.004909e+09,8.010410e+09
1,"autauga,alabama",2.061400e+06,2.040926e+06,2.015371e+06,2.097770e+06,1.983498e+06,2.144260e+06,2.072551e+06,2.052883e+06,2.033610e+06,1.973286e+06,1.984800e+06,2.044338e+06,2.146911e+06,2.150239e+06,2.054491e+06,2.065396e+06,2.062849e+06,2.003508e+06,1.937457e+06,2.120668e+06,2.023095e+06,1.968458e+06,2.068440e+06,2.078651e+06,2.024409e+06,2.071381e+06,2.090064e+06,2.056557e+06,1.962044e+06,2.030096e+06,1.993719e+06,2.068624e+06,1.995600e+06,2.034503e+06,1.986709e+06,2.021700e+06,2.046419e+06,1.967579e+06,2.069909e+06,...,2.024846e+06,1.989535e+06,2.121188e+06,2.059027e+06,2.048693e+06,1.962180e+06,1.944912e+06,2.077019e+06,2.036006e+06,2.038099e+06,1.984063e+06,2.004817e+06,2.057370e+06,2.013928e+06,2.041330e+06,2.124257e+06,2.059073e+06,2.038267e+06,2.036806e+06,2.060708e+06,1.956146e+06,2.015531e+06,1.993833e+06,2.047782e+06,1.970515e+06,1.955831e+06,2.009691e+06,2.087165e+06,1.997946e+06,1.991494e+06,2.102857e+06,2.051624e+06,2.063260e+06,2.082612e+06,2.052853e+06,2.020939e+06,2.139404e+06,2.085583e+06,2.025799e+06,2.027191e+06
2,"baldwin,alabama",6.092608e+06,6.032096e+06,5.956567e+06,6.200100e+06,5.862364e+06,6.337507e+06,6.125565e+06,6.067437e+06,6.010473e+06,5.832181e+06,5.866210e+06,6.042179e+06,6.345340e+06,6.355176e+06,6.072186e+06,6.104419e+06,6.096890e+06,5.921506e+06,5.726286e+06,6.267779e+06,5.979396e+06,5.817912e+06,6.113416e+06,6.143595e+06,5.983278e+06,6.122106e+06,6.177325e+06,6.078293e+06,5.798956e+06,6.000086e+06,5.892571e+06,6.113960e+06,5.898130e+06,6.013113e+06,5.871852e+06,5.975273e+06,6.048330e+06,5.815314e+06,6.117758e+06,...,5.984569e+06,5.880205e+06,6.269316e+06,6.085595e+06,6.055053e+06,5.799357e+06,5.748320e+06,6.138771e+06,6.017554e+06,6.023741e+06,5.864033e+06,5.925373e+06,6.080696e+06,5.952301e+06,6.033290e+06,6.278386e+06,6.085730e+06,6.024237e+06,6.019917e+06,6.090561e+06,5.781522e+06,5.957039e+06,5.892909e+06,6.052358e+06,5.823990e+06,5.780592e+06,5.939778e+06,6.168758e+06,5.905066e+06,5.885996e+06,6.215137e+06,6.063714e+06,6.098105e+06,6.155301e+06,6.067348e+06,5.973023e+06,6.323153e+06,6.164083e+06,5.987386e+06,5.991500e+06
3,"barbour,alabama",1.717072e+06,1.700017e+06,1.678731e+06,1.747366e+06,1.652182e+06,1.786091e+06,1.726360e+06,1.709977e+06,1.693923e+06,1.643676e+06,1.653266e+06,1.702859e+06,1.788299e+06,1.791071e+06,1.711316e+06,1.720400e+06,1.718278e+06,1.668850e+06,1.613831e+06,1.766440e+06,1.685165e+06,1.639654e+06,1.722936e+06,1.731441e+06,1.686259e+06,1.725385e+06,1.740947e+06,1.713037e+06,1.634312e+06,1.690996e+06,1.660695e+06,1.723089e+06,1.662262e+06,1.694668e+06,1.654856e+06,1.684003e+06,1.704593e+06,1.638922e+06,1.724159e+06,...,1.6